In [ ]:
using Distances

n = [256, 1024, 4096, 16384]
c = [0.3, 0.1, 0.03]
function Mor(n::Int64, c::Float64) 
    a=trunc(Int,sqrt(n))-1
    A=Vector{Float64}(undef,a+1)   ## grid : x-coord
    B=Vector{Float64}(undef,a+1)   ## grid : y-coord
    C=zeros(a+1,a+1)   ## Index by Morton order
    D=Matrix{Float64}(undef,n,n)
    for i=1:(a+1)
        A[i] = (i-1)/a 
        B[i] = 1-(i-1)/a
        for j=1:(a+1)
            C[i,j] = trunc(Int,(a+1)*(i-1)+j)
        end
    end

    for i=1:n
        for j=i:n
        D[i,j] = exp(-euclidean([A[findall(x->x==i,C)[1][1]], B[findall(x->x==i, C)[1][2]]],[A[findall(x->x==j,C)[1][1]], B[findall(x->x==j, C)[1][2]]])/c)
        D[j,i] = D[i,j]
        end
    end
    return D
end

using LinearAlgebra, Distributions, Statistics

function Blockreorder(n::Int64, c::Float64, ind::Array{Int64,1}, a::Array{Float64,1}, b::Array{Float64,1}, m::Int64)
    P=zeros(trunc(Int, n/m))
    for j=1:m:(n-m+1)
        subind=ind[j:(j+m-1)]
        A = Mor(n,c)[subind, subind]
        a0 = a[subind]
        b0 = b[subind]
        P[trunc(Int,j/m)+1] = RCMVN(Symmetric(A), a0, b0, 1, m)[1]  ## Required RCMVN, probability values for each m-sized blocks
    end
    ord=sortperm(P, rev=true)
    key=Array{Int64,1}(undef, n)
    for j=1:length(ord)
        key[(trunc(Int,length(ind)/length(P))*(j-1)+1):1:(trunc(Int,length(ind)/length(P))*j)]=[((ord[j]-1)*trunc(Int,length(ind)/length(P))+1) : 1 : (ord[j]*trunc(Int,length(ind)/length(P)));]   ## dimenstion arranging between vectors P and ind   ## dimenstion arranging between vectors P and ind
    end
    ind = ind[key]
    return ind
end

In [ ]:

iters = 20

m = 4; ns = 16; betas = 0.3  ## change betas to 0.1, 0.03 for alternative options
ans_1 = 0
time_1 = 0
    
a = fill(-Inf, ns)
b = zeros(ns)
ind=[1:1:ns;]

for p in 1:iters
    time_1 += @elapsed ( global ans_1 += Blockreorder(ns,betas,ind,a,b,m)[1])
end
    


time_1

In [ ]:
iters = 20

m = 4; ns = 64; betas = 0.3
ans_1 = 0
time_1 = 0
    
a = fill(-Inf, ns)
b = zeros(ns)
ind=[1:1:ns;]

for p in 1:iters
    time_1 += @elapsed ( global ans_1 += Blockreorder(ns,betas,ind,a,b,m)[1])
end
    


time_1

In [ ]:
iters = 20

m = 4; ns = 256; betas = 0.3
ans_1 = 0
time_1 = 0
    
a = fill(-Inf, ns)
b = zeros(ns)
ind=[1:1:ns;]

for p in 1:iters
    time_1 += @elapsed ( global ans_1 += Blockreorder(ns,betas,ind,a,b,m)[1])
end
    


time_1